## Observations and Insights 

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "Resources/Mouse_metadata.csv"
study_results_path = "Resources/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)
# Combine the data into a single dataset
big_df = mouse_metadata.merge(study_results,how='outer',on='Mouse ID')
# Display the data table for preview
big_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [6]:
# Checking the number of mice.
len(big_df['Mouse ID'].unique())

249

In [33]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# using the study results, because the merge won't allow the duplicate
duplicates = study_results.duplicated()

#seeing how many duplicates there are to determine how best proceed
duplicates.value_counts()


False    1892
True        1
dtype: int64

In [ ]:
#since there is only one duplicate, finding that ID is pretty straightforward
duplicated_index=duplicates.loc[idk==True].index
duplicate_mouse_id=study_results.iloc[137,0]

In [31]:
# Optional: Get all the data for the duplicate mouse ID. 

duplicated_index=duplicates.loc[idk==True].index
duplicate_mouse_id=study_results.iloc[137,0]
list_of_dupe = big_df.loc[big_df['Mouse ID']=='g989'].index.tolist()
big_df.loc[list_of_dupe,:]
#here is all the data with the mouse id that was duplicated. as stated two cells previous, the merge didn't create a duplicate!

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
912,g989,Propriva,Female,21,26,10,51.745156,0
914,g989,Propriva,Female,21,26,15,51.325852,1
916,g989,Propriva,Female,21,26,20,55.326122,1
918,g989,Propriva,Female,21,26,25,56.045564,1
919,g989,Propriva,Female,21,26,30,59.082294,1
920,g989,Propriva,Female,21,26,35,62.570880,2


In [10]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# if there was a duplicate in the dataframe, we would remove it this way
# I think we would have gotten duplicates if we merged with the results on the left (i used the metadata on the left, since it 'should' have been unique)
big_df = big_df.drop_duplicates(subset={'Mouse ID','Timepoint'},keep='first')

In [11]:
# Checking the number of mice in the clean DataFrame.
len(big_df['Mouse ID'].unique())

#this number is the same because we called unique both times; which would have ignored the duplicate anyways

249

## Summary Statistics

In [44]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.
regimen_groups = big_df.groupby('Drug Regimen')
mean = regimen_groups['Tumor Volume (mm3)'].mean()
median = regimen_groups['Tumor Volume (mm3)'].median()
std = regimen_groups['Tumor Volume (mm3)'].std()
pd.DataFrame(regimen_groups['Tumor Volume (mm3)'])
#SEM = st.sem(regimen_groups['Tumor Volume (mm3)'])


AttributeError: 'SeriesGroupBy' object has no attribute 'value'

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
